In [1]:
import os

import numpy as np
import gymnasium as gym

import gym_env

In [10]:
# Construct the environment
env = gym.make("simple-5x5")
env.reset()

/Users/abizzle/anaconda3/envs/gym/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


({'agent': array([0, 0]), 'target': array([4, 4])}, {'distance': 8.0})

In [11]:
actions = np.arange(env.action_space.n, dtype=int)
start_loc = env.unwrapped.start_loc
target_loc = env.unwrapped.target_loc
maze = env.unwrapped.maze
size = maze.size

In [12]:
print(f"actions: {actions}, start loc: {start_loc}, target loc: {target_loc}, size: {size}")

actions: [0 1 2 3], start loc: [0 0], target loc: [4 4], size: 25


In [13]:
def row_col_to_index(row, col, len):
    """
    Converts (row,col) to an index in array
    """
    return row*len + col

In [29]:
# Get the transition matrix T N^2 x N^2
T = np.zeros(shape=(size, size))

# loop through the maze
for row in range(maze.shape[0]):
    for col in range(maze.shape[1]):
        # at each location, we want to store the location, keep track of which new states we transition into, and how many states we transition into
        loc = np.array((row,col))
        new_states = []
        for action in actions:     # loop through actions
            env.unwrapped.agent_loc = loc                  # set new agent location based on where we are in maze
            obs, reward, term, _, _ = env.step(action)     # take action

            # if dont move because we hit a boundary, do nothing
            if (obs['agent'] == loc).all():
                continue
            new_states.append(obs['agent'])
        
        idx_cur = row_col_to_index(row, col, maze.shape[0]-1)
        for new_state in new_states:
            idx_new = row_col_to_index(new_state[0], new_state[1], maze.shape[0]-1)
            T[idx_cur, idx_new] = 1/len(new_states)

In [30]:
T

array([[0.        , 0.5       , 0.        , 0.        , 0.5       ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.5       , 0.        , 0.5       , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.5       , 0.        , 0.5       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.